In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
spark=SparkSession.builder.appName('Struct Type').getOrCreate()

In [3]:
raw_data = [
    ("U001","Abhishek",28,"Hyderabad",50000),
    ("U002","Neha",32,"Delhi",62000),
    ("U003","Ravi",25,"Bangalore",45000),
    ("U004","Pooja",29,"Mumbai",58000)
]

In [4]:
from pyspark.sql.types import(
    StructType,
    StructField,
    StringType,
    IntegerType,
    LongType
)

In [5]:
user_schema=StructType([
    StructField('user_id',StringType(),nullable=False),
    StructField('name',StringType(),nullable=True),
    StructField('age',IntegerType(),nullable=True),
    StructField('city',StringType(),nullable=True),
    StructField('salary',LongType(),nullable=True)
])

In [7]:
df_users=spark.createDataFrame(data=raw_data,schema=user_schema)
df_users.show()
df_users.printSchema()

+-------+--------+---+---------+------+
|user_id|    name|age|     city|salary|
+-------+--------+---+---------+------+
|   U001|Abhishek| 28|Hyderabad| 50000|
|   U002|    Neha| 32|    Delhi| 62000|
|   U003|    Ravi| 25|Bangalore| 45000|
|   U004|   Pooja| 29|   Mumbai| 58000|
+-------+--------+---+---------+------+

root
 |-- user_id: string (nullable = false)
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- salary: long (nullable = true)



In [ ]:
# wrong data
raw_data2=[
    ("U005","Amit","Thirty","Chennai",40000)
]
df_users=spark.createDataFrame(data=raw_data2,schema=user_schema)
df_users.show()

# Array Type

In [9]:
from pyspark.sql.types import ArrayType

In [10]:
interest_data = [
    ("U001",["AI","ML","Cloud"]),
    ("U002",["Testing","Automation"]),
    ("U003",["Data Engineering","Spark","Kafka"]),
    ("U004",["UI/UX"])
]

In [11]:
#from IPython.utils.ipstruct import Struct
interest_schema=StructType([
    StructField('user_id',StringType(),nullable=False),
    StructField('interests',ArrayType(StringType()),nullable=True)
])

In [12]:
df_interests=spark.createDataFrame(data=interest_data,schema=interest_schema)
df_interests.show(truncate=False)
df_interests.printSchema()

+-------+--------------------------------+
|user_id|interests                       |
+-------+--------------------------------+
|U001   |[AI, ML, Cloud]                 |
|U002   |[Testing, Automation]           |
|U003   |[Data Engineering, Spark, Kafka]|
|U004   |[UI/UX]                         |
+-------+--------------------------------+

root
 |-- user_id: string (nullable = false)
 |-- interests: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [15]:
from pyspark.sql.functions import explode
df_interests.select("user_id",explode("interests").alias("interest")).show()

+-------+----------------+
|user_id|        interest|
+-------+----------------+
|   U001|              AI|
|   U001|              ML|
|   U001|           Cloud|
|   U002|         Testing|
|   U002|      Automation|
|   U003|Data Engineering|
|   U003|           Spark|
|   U003|           Kafka|
|   U004|           UI/UX|
+-------+----------------+



# Map Type

In [18]:
from pyspark.sql.types import MapType

In [19]:
device_data = [
    ("U001",{"mobile":120,"laptop":300}),
    ("U002",{"tablet":80}),
    ("U003",{"mobile":200,"desktop":400}),
    ("U004",{"laptop":250})
]

In [20]:
device_schema=StructType([
    StructField('user_id',StringType(),nullable=False),
    StructField('device_usage',MapType(StringType(),IntegerType()),nullable=True)
])

In [21]:
df_devices=spark.createDataFrame(data=device_data,schema=device_schema)
df_devices.show(truncate=False)
df_devices.printSchema()

+-------+-------------------------------+
|user_id|device_usage                   |
+-------+-------------------------------+
|U001   |{mobile -> 120, laptop -> 300} |
|U002   |{tablet -> 80}                 |
|U003   |{mobile -> 200, desktop -> 400}|
|U004   |{laptop -> 250}                |
+-------+-------------------------------+

root
 |-- user_id: string (nullable = false)
 |-- device_usage: map (nullable = true)
 |    |-- key: string
 |    |-- value: integer (valueContainsNull = true)



In [22]:
nested_data = [
    ("U001",("Hyderabad","Telangana",500081)),
    ("U002",("Delhi","Delhi",110001)),
    ("U003",("Bangalore","Karnataka",560001))
]

In [23]:
address_schema=StructType([
    StructField('city',StringType(),nullable=True),
    StructField('state',StringType(),nullable=True),
    StructField('pincode',IntegerType(),nullable=True)
])

profile_schema=StructType([
    StructField('user_id',StringType(),nullable=False),
    StructField('address',address_schema,nullable=True)
])

In [24]:
df_profiles=spark.createDataFrame(data=nested_data,schema=profile_schema)
df_profiles.show(truncate=False)
df_profiles.printSchema()

+-------+------------------------------+
|user_id|address                       |
+-------+------------------------------+
|U001   |{Hyderabad, Telangana, 500081}|
|U002   |{Delhi, Delhi, 110001}        |
|U003   |{Bangalore, Karnataka, 560001}|
+-------+------------------------------+

root
 |-- user_id: string (nullable = false)
 |-- address: struct (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- state: string (nullable = true)
 |    |-- pincode: integer (nullable = true)



In [25]:
df_profiles.select(
    "user_id",
    "address.city",
    "address.state",
    "address.pincode"
).show()

+-------+---------+---------+-------+
|user_id|     city|    state|pincode|
+-------+---------+---------+-------+
|   U001|Hyderabad|Telangana| 500081|
|   U002|    Delhi|    Delhi| 110001|
|   U003|Bangalore|Karnataka| 560001|
+-------+---------+---------+-------+



In [26]:
from pyspark.sql.functions import col
df_users.withColumn(
    "salary_int",
    col("salary").cast("int")
)

DataFrame[user_id: string, name: string, age: int, city: string, salary: bigint, salary_int: int]